# Giriş işlemleri

In [ ]:
%%capture
!pip install trl bitsandbytes

## Kütüphane içe aktarma

In [ ]:
import os
import pandas as pd
import torch
from trl import SFTTrainer, SFTConfig
from datasets import Dataset
from typing import List, Literal, Tuple, Any, Dict
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig, PreTrainedTokenizer)
from peft import LoraConfig, get_peft_model

In [ ]:
def is_colab() -> bool:
    """
    Google Colab ortamında çalışıp çalışmadığını kontrol eden fonksiyon.

    Args:
        None

    Returns:
        bool: Eğer kod Google Colab'da çalışıyorsa True, aksi halde False döndürür
    """
    try:
        import google.colab
        return True
    except ImportError:
        return False

In [ ]:
# Kök dizin belirleme
if is_colab():
    """
    Eğer kod Google Colab ortamında çalışıyorsa, Google Drive'ı bağlar ve
    kök dizini Google Drive içindeki "turkish_gpt2_finetuning" klasörü olarak ayarlar.
    """
    from google.colab import drive
    drive.mount('/content/drive')  # Google Drive'ı Colab ortamına bağlar
    kok_dizin = "/content/drive/MyDrive/turkish_gpt2_finetuning"  # Drive içindeki çalışma klasörünü belirler
else:
    """
    Eğer kod yerel bir ortamda çalışıyorsa, kök dizini mevcut çalışma dizini olarak ayarlar.
    """
    kok_dizin = os.getcwd()  # Mevcut çalışma dizinini alır

# Belirlenen kök dizini kullanıcıya bilgi olarak gösterir
print(f"Kök dizin: {kok_dizin}\n Not: eğer colab kullanıyorsanız, dizini değiştirmeniz gerekir.")

## Veri kümesi biçimlendirme, yol ve model yolu tanımlamaları

In [ ]:
# Veri kümesi yolu ve karıştırma seed değeri tanımlamaları
veri_kumesi_yolu = os.path.join(kok_dizin, "soru_cevap.csv")
veri_kumesi_karistirma = 571  # Karıştırmada kullanılacak sabit seed değeri
sonuc_dizini = os.path.join(kok_dizin, "sonuclar")  # Sonuçların kaydedileceği dizin

# Model kaydetme dizinleri
# gpt4o verisiyle eğitilmiş modeller
gpt2_medium_kaydetme_dizini_gpt4o = os.path.join(kok_dizin, "gpt2_medium_gpt4o")
gpt2_large_kaydetme_dizini_gpt4o = os.path.join(kok_dizin, "gpt2_large_gpt4o")

# deepseek verisiyle eğitilmiş modeller
gpt2_medium_kaydetme_dizini_deepseek = os.path.join(kok_dizin, "gpt2_medium_deepseek")
gpt2_large_kaydetme_dizini_deepseek = os.path.join(kok_dizin, "gpt2_large_deepseek")

# Model adı tanımlamaları
gpt2_medium_model_adi = "ytu-ce-cosmos/turkish-gpt2-medium"
gpt2_large_model_adi = "ytu-ce-cosmos/turkish-gpt2-large"

# Veri kümesi sütun tanımlamaları
soru_sutunu = "Soru"
gpt4o_cevap_sutunu = "gpt4o cevabı"
deepseek_cevap_sutunu = "deepseek cevabı"

# Özel token tanımlamaları (eğitim formatı için)
soru_baslangic = "<SORU>"
soru_bitis = "</SORU>"
cevap_baslangic = "<CEVAP>"
cevap_bitis = "</CEVAP>"
ornek_bitis = "<|endoftext|>"  # GPT-2'nin EOS tokeni

# Tokenizer'a eklenecek özel tokenler listesi
ozel_tokenler = [soru_baslangic, soru_bitis, cevap_baslangic, cevap_bitis]

## Veri kümesi okuma

In [ ]:
# -----------------------------------------------
# CSV dosyasını okuma ve gerekli sütunları ayıklama
# -----------------------------------------------

try:
    # -------------------------------------------------
    # 1) CSV dosyasını diskteki yolundan (veri_kumesi_yolu)
    #    pandas DataFrame olarak içeri aktar.
    # -------------------------------------------------
    df = pd.read_csv(veri_kumesi_yolu)

    # -------------------------------------------------
    # 2) Kullanıcıdan/konfigürasyondan gelen beklenen
    #    sütun isimlerini bir liste hâlinde tanımla.
    #
    #    Burada:
    #      - "Soru" : soruları içeren sütun
    #      - gpt4o_cevap_sutunu : GPT-4o modelinin cevapları
    #      - deepseek_cevap_sutunu : DeepSeek modelinin cevapları
    # -------------------------------------------------
    gereken_sutunlar = ["Soru", gpt4o_cevap_sutunu, deepseek_cevap_sutunu]

    # -------------------------------------------------
    # 3) Her bir gerekli sütunun DataFrame’de mevcut
    #    olup olmadığını kontrol et. Eksikse uyarı ver.
    # -------------------------------------------------
    for sutun in gereken_sutunlar:
        if sutun not in df.columns:
            print(f"'{sutun}' sütunu veri kümesinde bulunamadı!")

    # -------------------------------------------------
    # 4) Sadece gerekli sütunları içeren yeni bir
    #    DataFrame (soru_cevap_df) oluştur.
    # -------------------------------------------------
    soru_cevap_df = df[gereken_sutunlar]

    # -------------------------------------------------
    # 5) Doğrulama amacıyla ilk 5 satırı ekrana yazdır.
    # -------------------------------------------------
    print("Veri kümesinin ilk birkaç satırı:")
    print(soru_cevap_df.head(5))

    # -------------------------------------------------
    # 6) Toplam kaç soru-cevap çifti olduğunu bildir.
    # -------------------------------------------------
    print(f"\nToplam {len(soru_cevap_df)} soru-cevap çifti bulundu.")

# -------------------------------------------------
# 7) Dosya bulunamazsa kullanıcıyı bilgilendir.
# -------------------------------------------------
except FileNotFoundError:
    print(f"'{veri_kumesi_yolu}' dosyası bulunamadı. Lütfen dosya yolunu kontrol ediniz.")

# -------------------------------------------------
# 8) Diğer tüm hataları genel başlıkta yakala
#    ve mesajını göster.
# -------------------------------------------------
except Exception as e:
    print(f"Veri okuma hatası: {e}")

In [ ]:
# ----------------------------------------------
# Veri kümesini karıştırma (shuffle) işlemi
# ----------------------------------------------

# 1) sample(frac=1) metodu:
#    • frac=1 -> DataFrame’in tamamını örnekle, yani tüm satırları rastgele sıraya diz.
#    • random_state=veri_kumesi_karistirma ->
#      Aynı seed değeriyle her çalıştırmada aynı karışık sıralamayı elde
#      etmeyi sağlar (deneysellik için tekrar üretilebilirlik) [[2]].
shuffled_df = soru_cevap_df.sample(
    frac=1,                     # Tüm satırları al
    random_state=veri_kumesi_karistirma  # Sabit tohum (seed)
)

# ----------------------------------------------
# İndeksleri sıfırlama
# ----------------------------------------------
# 2) Karıştırma sonrası eski indeksler anlamını yitirir.
#    reset_index(drop=True) ile:
#      • drop=True   -> Eski indeks sütunu DataFrame’e eklenmez,
#                       doğrudan atılır.
#      • Yeni indeksler 0..N-1 şeklinde oluşturulur.
shuffled_df = shuffled_df.reset_index(drop=True)

# ----------------------------------------------
# Karıştırılmış veri kümesini gözlemleme
# ----------------------------------------------
# 3) İlk 5 satırı yazdırarak karıştırmanın
#    başarıyla gerçekleştiğini doğrula.
print("Karıştırılmış veri kümesinin ilk birkaç satırı:")
print(shuffled_df.head(5))

# 4) Toplam satır sayısını ekrana yazdır.
print(f"\nKarıştırılmış veri kümesi boyutu: {len(shuffled_df)} satır")


# Veri kümesi işleme

## Veri kümesi biçimlendirme fonksiyonu

In [ ]:
def veri_kumesini_egitim_formatina_donustur(
    df              : pd.DataFrame,   # Soru–cevap verilerini tutan pandas DataFrame
    cevap_sutunu    : str,            # Hangi sütunun “cevap” olarak kullanılacağını belirtir
    *,
    soru_baslangic  : str = "<SORU>",      # Soru başlangıcını işaretleyen özel etiket
    soru_bitis      : str = "</SORU>",     # Soru bitişini   işaretleyen özel etiket
    cevap_baslangic : str = "<CEVAP>",     # Cevap başlangıcını işaretleyen özel etiket
    cevap_bitis     : str = "</CEVAP>",    # Cevap bitişini   işaretleyen özel etiket
    ornek_bitis     : str = "<|endoftext|>"# Her örneğin sonunda yer alacak EOS biçimindeki etiket
) -> List[str]:
    """
    Soru–cevap veri kümesini GPT-2 eğitimine uygun tekil metin dizilerine dönüştürür.

    Fonksiyon, her satır (soru-cevap çifti) için aşağıdaki genel formata sahip
    birleştirilmiş bir string üretir ve çıkan tüm örnekleri liste hâlinde döndürür:

        <SORU>  {soru_metni}  </SORU>  <CEVAP>  {cevap_metni}  </CEVAP><|endoftext|>

    NOT: Yalnızca metin birleştirmesi yapar, herhangi bir tokenizasyon işlemi uygulamaz.

    Parametreler
    ------------
    df : pandas.DataFrame
        “Soru” ve istenen cevap sütununu içeren veri çerçevesi.
    cevap_sutunu : str
        Cevapların bulunduğu sütun adı (örn. "gpt4o cevabı").
    soru_baslangic : str, varsayılan "<SORU>"
        Soru başlangıcı için kullanılacak özel belirteç.
    soru_bitis : str, varsayılan "</SORU>"
        Soru bitişi   için kullanılacak özel belirteç.
    cevap_baslangic : str, varsayılan "<CEVAP>"
        Cevap başlangıcı için kullanılacak özel belirteç.
    cevap_bitis : str, varsayılan "</CEVAP>"
        Cevap bitişi   için kullanılacak özel belirteç.
    ornek_bitis : str, varsayılan "<|endoftext|>"
        Her örneğin sonunda eklenecek bitiş/EOS belirteci.

    Dönüş Değeri
    ------------
    List[str]
        Eğitimde doğrudan kullanılmaya hazır, “birleştirilmiş” metin dizilerinin listesi.
    """

    # ---------------------------------------------------------------------------- #
    # 1) Çıktıları tutmak için boş bir liste oluşturuyoruz.                        #
    #    Her iterasyonda, yeni biçimlendirilmiş metin bu listeye eklenecek.        #
    # ---------------------------------------------------------------------------- #
    egitim_metinleri: List[str] = []

    # ---------------------------------------------------------------------------- #
    # 2) DataFrame'deki her satır (index, satir) ikilisi üzerinde döngü kuruyoruz. #
    #    • _      : index değeri (kullanılmadığı için alt tire)                    #
    #    • satir  : pandas.Series, o anki satıra ait veriler                       #
    # ---------------------------------------------------------------------------- #
    for _, satir in df.iterrows():

        # Satırdan “Soru” sütununu alıp baştaki/sondaki boşlukları temizliyoruz.
        soru: str = satir["Soru"].strip()

        # Aynı işlemi cevap sütunu için de yapıyoruz (gönderilen isim dinamik).
        cevap: str = satir[cevap_sutunu].strip()

        # -------------------------------------------------------------------- #
        # 3) Soru ve cevabı istenen etiketlerle sarmalayıp tek bir string      #
        #    oluşturuyoruz. Yapı:                                              #
        #      <SORU> Soru </SORU> <CEVAP> Cevap </CEVAP><|endoftext|>         #
        # -------------------------------------------------------------------- #
        bicimlendirilmis_metin: str = (
            f"{soru_baslangic} {soru} {soru_bitis} "
            f"{cevap_baslangic} {cevap} {cevap_bitis}"
            f"{ornek_bitis}"
        )

        # 4) Elde edilen string’i çıktı listesine ekliyoruz.
        egitim_metinleri.append(bicimlendirilmis_metin)

    # ---------------------------------------------------------------------------- #
    # 5) Özet bilgi: toplam kaç örnek üretildiğini ve şablonun nasıl göründüğünü   #
    #    kullanıcıya yazdırıyoruz.                                                #
    # ---------------------------------------------------------------------------- #
    print(f"Toplam {len(egitim_metinleri)} adet eğitim örneği oluşturuldu.")
    print(
        f"Örnek biçimi: "
        f"{soru_baslangic} [Soru] {soru_bitis} "
        f"{cevap_baslangic} [Cevap] {cevap_bitis}{ornek_bitis}"
    )

    # 6) Hazırlanan tüm metinleri geri döndürüyoruz.
    return egitim_metinleri

## Veri kümelerini işle ve oluştur

In [ ]:
veri_kumesi_gpt4o = veri_kumesini_egitim_formatina_donustur(shuffled_df, gpt4o_cevap_sutunu)
veri_kumesi_deepseek = veri_kumesini_egitim_formatina_donustur(shuffled_df, deepseek_cevap_sutunu)

# Eğitim işlemleri

## Model Yükleme ve Eğitme Fonksiyonları

### Eğitim argümanları oluşturma fonksiyonu

In [ ]:
import math
max_seq_length = 1024   # üstte sabit tanım

def training_arguments_getir(
    kaydetme_dizin      : str,
    veri_kumesi_uzunlugu: int,
    *,
    learning_rate : float = 1e-4,
    batch_size    : int   = 2,
    grad_accum    : int   = 32,
    epochs        : int   = 1,
    max_seq_length: int   = 1024,
    save_per_epoch: int   = 4,
    log_per_epoch : int   = 12,
) -> SFTConfig:
    """
    TRL kütüphanesinin `SFTConfig` sınıfını kolayca oluşturmak için
    öntanımlı hiper-parametrelerle donatılmış yardımcı fonksiyon.

    Args:
        kaydetme_dizin (str): Checkpoint ve log dosyalarının kaydedileceği dizin.
        veri_kumesi_uzunlugu (int): Toplam örnek (satır) sayısı.
        learning_rate (float, optional): Öğrenme oranı. Varsayılan 1e-4.
        batch_size (int, optional): GPU başına batch büyüklüğü. Varsayılan 2.
        grad_accum (int, optional): Gradient accumulation adım sayısı.
        epochs (int, optional): Eğitim süresi (epoch). Varsayılan 1.
        max_seq_length (int, optional): Girdi dizisi tokensel azami uzunluk.
        save_per_epoch (int, optional): Bir epoch kaç parçaya bölünerek
            checkpoint alınacağını belirler. Örn. 4 → epoch/4 adımda bir.
        log_per_epoch (int, optional): Bir epoch kaç parçaya bölünerek
            loss değeri loglanacağını belirler.
    Returns:
        SFTConfig: TRL/Trainer ile uyumlu, ön ayarlı konfigürasyon nesnesi.
    """

    # ----------------- Türetilmiş değerleri hesapla ----------------------- #
    ngpu: int = max(torch.cuda.device_count(), 1)         # GPU adedi
    effective_batch: int = batch_size * grad_accum * ngpu # Toplam güncel batch
    steps_per_epoch: int = math.ceil(veri_kumesi_uzunlugu / effective_batch)
    max_steps: int        = epochs * steps_per_epoch
    save_steps: int       = max(1, steps_per_epoch // save_per_epoch)
    logging_steps: int    = max(1, steps_per_epoch // log_per_epoch)

    print(
        f"[INFO] veri: {veri_kumesi_uzunlugu} örnek | gpu: {ngpu} | "
        f"effective_batch: {effective_batch} | steps/epoch: {steps_per_epoch} | "
        f"total steps: {max_steps}"
    )

    # ----------------- SFTConfig için argüman sözlüğü --------------------- #
    cfg_kwargs: Dict[str, Any] = dict(
        output_dir = kaydetme_dizin,

        # Temel hiper-parametreler
        learning_rate               = learning_rate,
        per_device_train_batch_size = batch_size,
        gradient_accumulation_steps = grad_accum,

        # Sekuans/eos ayarları
        max_seq_length = max_seq_length,

        # Optimizasyon
        optim             = "paged_adamw_8bit",
        weight_decay      = 0.1,
        warmup_ratio      = 0.1,
        lr_scheduler_type = "cosine",
        max_grad_norm     = 0.1,

        # Loglama
        logging_dir      = os.path.join(kaydetme_dizin, "logs"),
        logging_strategy = "steps",
        logging_steps    = logging_steps,
        report_to        = "tensorboard",
        run_name         = os.path.basename(kaydetme_dizin),

        # Checkpoint
        save_strategy = "steps",
        save_steps    = save_steps,

        # Epoch tabanlı çalışma
        num_train_epochs = epochs,
        max_steps        = -1,  # -1 → Trainer epoch modunda çalış
    )

    # Nesneyi oluşturup geri döndür
    return SFTConfig(**cfg_kwargs)

### Model Yükleme fonksiyonu

In [ ]:
def model_ve_tokenizer_yukle(
    model_adi      : str,
    max_seq_length : int                           = max_seq_length,
    lora_rank      : int                           = 16,
    random_state   : int                           = veri_kumesi_karistirma,
    target_modules : List[str]                     = ("c_attn", "c_proj", "c_fc"),
    ozel_tokenler  : List[str]                     = ozel_tokenler,
    quant_mode     : Literal["4bit", "8bit", "16bit"] = "16bit",
) -> Tuple[torch.nn.Module, PreTrainedTokenizer]:
    """
    İstenen GPT-2 tabanlı modeli; LoRA adaptörleri ve (opsiyonel) 4/8-bit
    quantization ile belleğe yükler. Ayrıca tokenizer’ı hazırlar.

    Args:
        model_adi (str): Hugging Face model adı veya yol.
        max_seq_length (int, optional): Modelin `max_position_embeddings` değeri.
        lora_rank (int, optional): LoRA rank değeri (R). Varsayılan 16.
        random_state (int, optional): CUDA random seed.
        target_modules (List[str], optional): LoRA uygulanacak katman adları.
        ozel_tokenler (List[str] | None): Ek special token listesi.
        quant_mode (Literal), optional: "4bit", "8bit" veya "16bit".
            • "4bit": QLoRA nf4
            • "8bit": bitsandbytes 8-bit
            • "16bit": Tam presizyon (fp16/bf16)
    Returns:
        Tuple[torch.nn.Module, PreTrainedTokenizer]: (model, tokenizer)
    """

    # ---------------- Tokenizer hazırlığı --------------------------------- #
    tokenizer: PreTrainedTokenizer = AutoTokenizer.from_pretrained(
        model_adi, use_fast=True
    )
    # Sağ tarafta pad / truncate
    tokenizer.padding_side, tokenizer.truncation_side = "right", "right"

    # Model pad_token tanımlı değilse EOS’u pad olarak ekle
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({"pad_token": tokenizer.eos_token})

    # ---------------- Quantization / dtype seçimi ------------------------- #
    quant_cfg: BitsAndBytesConfig | None = None
    model_kwargs: Dict[str, Any] = dict(device_map="auto", trust_remote_code=True)

    if quant_mode == "4bit":
        # QLoRA konfigürasyonu
        quant_cfg = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype = (
                torch.bfloat16 if torch.cuda.get_device_capability(0)[0] >= 8
                else torch.float16
            ),
        )
        model_kwargs["quantization_config"] = quant_cfg

    elif quant_mode == "8bit":
        quant_cfg = BitsAndBytesConfig(load_in_8bit=True)
        model_kwargs["quantization_config"] = quant_cfg

    else:  # "16bit"
        model_kwargs["torch_dtype"] = (
            torch.bfloat16 if torch.cuda.get_device_capability(0)[0] >= 8
            else torch.float16
        )

    # ---------------- Modeli yükle ---------------------------------------- #
    model = AutoModelForCausalLM.from_pretrained(model_adi, **model_kwargs)

    # Konfig ayarları
    model.config.max_position_embeddings = max_seq_length
    model.config.use_cache = False
    model.gradient_checkpointing_enable()  # bellek tasarrufu

    # ---------------- LoRA adaptörü ekle ---------------------------------- #
    lora_cfg: LoraConfig = LoraConfig(
        r = lora_rank,
        lora_alpha = lora_rank,
        target_modules = list(target_modules),
        bias = "none",
        task_type = "CAUSAL_LM",
        fan_in_fan_out = True,
    )
    model = get_peft_model(model, lora_cfg)
    model.print_trainable_parameters()

    # ---------------- Özel token’lar ekle --------------------------------- #
    if ozel_tokenler:
        tokenizer.add_special_tokens({"additional_special_tokens": ozel_tokenler})
        model.resize_token_embeddings(len(tokenizer))

    # Rastgeleliği sabitle
    torch.cuda.manual_seed(random_state)
    print(f"{model_adi} modeli '{quant_mode}' kipinde yüklendi (+LoRA).")

    return model, tokenizer

### Eğitici Getirme fonksiyonu

In [ ]:
def trainer_getir(
    model          : torch.nn.Module,
    training_args  : SFTConfig,
    veri_kumesi    : List[str],
    tokenizer      : PreTrainedTokenizer
) -> SFTTrainer:
    """
    Metin listesini tokenize edip `SFTTrainer` için PyTorch
    tensörlerine dönüştürür ve eğitici nesnesini oluşturur.

    Args:
        model (torch.nn.Module): PEFT (LoRA) ile zenginleştirilmiş GPT-2 modeli.
        training_args (SFTConfig): `training_arguments_getir` çıktısı.
        veri_kumesi (List[str]): Önceden formatlanmış eğitim metinleri listesi.
        tokenizer (PreTrainedTokenizer): Aynı modele ait tokenizer.

    Returns:
        SFTTrainer: TRL eğitici nesnesi.
    """

    # 1) Tokenizasyon → padding + truncation (max_length parametresi zaten var)
    toks = tokenizer(
        veri_kumesi,
        padding="max_length",
        truncation=True,
        max_length=training_args.max_seq_length,
    )

    # 2) Tensör formatına dönüştür
    data_dict: Dict[str, List[List[int]]] = {
        "input_ids":      toks["input_ids"],
        "attention_mask": toks["attention_mask"],
    }
    train_dataset: Dataset = Dataset.from_dict(data_dict)
    train_dataset.set_format(type="torch")

    print(f"Eğitim için {len(train_dataset)} örnek hazırlandı.")

    # 3) SFTTrainer oluştur
    trainer: SFTTrainer = SFTTrainer(
        model         = model,
        args          = training_args,
        train_dataset = train_dataset,
    )
    return trainer

### Veri kümesi özelinde model eğitme fonksiyonu

In [ ]:
from transformers.trainer_utils import get_last_checkpoint
import json

def model_egit_ve_kaydet(
    model                : torch.nn.Module,
    tokenizer            : PreTrainedTokenizer,
    veri_kumesi          : List[str],
    model_kaydetme_dizini: str,
) -> None:
    """
    LoRA-adaptörlü modeli verilen veri kümesiyle eğitir ve çıktı/ checkpoint
    dizinini yönetir. Eğer dizinde geçerli bir checkpoint varsa devam eder,
    yoksa eğitimi sıfırdan başlatır.

    Args:
        model (torch.nn.Module): Eğitilecek model.
        tokenizer (PreTrainedTokenizer): İlgili tokenizer.
        veri_kumesi (List[str]): Eğitim örnekleri listesi.
        model_kaydetme_dizini (str): Checkpoint / log dizini yolu.

    Returns:
        None
    """

    # ------------------------------------------------------------------ #
    # 0) Çıktı dizinini oluştur (varsa dokunma)                          #
    # ------------------------------------------------------------------ #
    os.makedirs(model_kaydetme_dizini, exist_ok=True)

    # ------------------------------------------------------------------ #
    # 1) Eğitim argümanlarını hazırla                                    #
    # ------------------------------------------------------------------ #
    training_args: SFTConfig = training_arguments_getir(
        kaydetme_dizin       = model_kaydetme_dizini,
        veri_kumesi_uzunlugu = len(veri_kumesi),
    )

    # ------------------------------------------------------------------ #
    # 2) Trainer oluştur                                                #
    # ------------------------------------------------------------------ #
    trainer: SFTTrainer = trainer_getir(
        model         = model,
        training_args = training_args,
        veri_kumesi   = veri_kumesi,
        tokenizer     = tokenizer,
    )

    # ------------------------------------------------------------------ #
    # 3) Checkpoint araması                                              #
    #    transformers.trainer_utils.get_last_checkpoint()               #
    #    - Geçerli bir checkpoint klasörü bulursa yolunu döndürür        #
    #    - Hiç bulamazsa `None` döndürür                                 #
    # ------------------------------------------------------------------ #
    checkpoint_yolu: str | None = None
    if os.path.isdir(model_kaydetme_dizini):
        checkpoint_yolu = get_last_checkpoint(model_kaydetme_dizini)

    if checkpoint_yolu:
        print(f"[Eğitim] Geçerli checkpoint bulundu: {checkpoint_yolu}\n"
              f"         Eğitime kaldığı yerden devam ediliyor...")
    else:
        print("[Eğitim] Geçerli checkpoint bulunamadı. Eğitim sıfırdan başlıyor...")

    # ------------------------------------------------------------------ #
    # 4) Eğitim döngüsü                                                  #
    #    • Checkpoint varsa resume_from_checkpoint ile devam et          #
    #    • Hata durumunda yakalayıp temiz bir başlangıca geç             #
    # ------------------------------------------------------------------ #
    try:
        trainer.train(resume_from_checkpoint=checkpoint_yolu)
    except (ValueError, FileNotFoundError) as e:
        # Checkpoint bozuk / eksik dosya → temiz başlangıç
        print(f"[Uyarı] Checkpoint okunamadı ({e}). "
              f"Eğitim sıfırdan tekrar başlatılıyor...")
        # Dilerseniz sorunlu checkpoint klasörünü silebilirsiniz:
        trainer.train()  # sıfırdan

    # ------------------------------------------------------------------ #
    # 5) Model, tokenizer ve logları kaydet                              #
    # ------------------------------------------------------------------ #
    trainer.save_model(model_kaydetme_dizini)
    tokenizer.save_pretrained(model_kaydetme_dizini)

    log_json_path: str = os.path.join(model_kaydetme_dizini, "training_log.json")
    with open(log_json_path, "w", encoding="utf-8") as f:
        json.dump(trainer.state.log_history, f, ensure_ascii=False, indent=4)

    print(f"[Eğitim] Model ve tokenizer '{model_kaydetme_dizini}' dizinine kaydedildi.")

    # ------------------------------------------------------------------ #
    # 6) Bellek temizliği                                               #
    # ------------------------------------------------------------------ #
    del model
    torch.cuda.empty_cache()


## GPT-2 Medium modelini GPT-4o veri kümesi ile eğit

In [ ]:
print("\nGPT-2 Medium modeli GPT-4o veri kümesi ile eğitiliyor...")
model_medium, tokenizer_medium = model_ve_tokenizer_yukle(gpt2_medium_model_adi)
# GPT-4o veri kümesiyle Medium modeli eğit
model_egit_ve_kaydet(
    model_medium,
    tokenizer_medium,
    veri_kumesi_gpt4o,
    gpt2_medium_kaydetme_dizini_gpt4o,
)

## GPT-2 Medium modelini DeepSeek veri kümesi ile eğit

In [ ]:
# DeepSeek veri kümesiyle Medium modeli eğit
print("\nGPT-2 Medium modeli DeepSeek veri kümesi ile eğitiliyor...")
model_medium, tokenizer_medium = model_ve_tokenizer_yukle(gpt2_medium_model_adi)
model_egit_ve_kaydet(
    model_medium,
    tokenizer_medium,
    veri_kumesi_deepseek,
    gpt2_medium_kaydetme_dizini_deepseek,
)

## GPT-2 Large modelini GPT-4o veri kümesi ile eğit

In [ ]:
# GPT-4o veri kümesiyle Large modeli eğit
print("\nGPT-2 Large modeli GPT-4o veri kümesi ile eğitiliyor...")
model_large, tokenizer_large = model_ve_tokenizer_yukle(gpt2_large_model_adi)
model_egit_ve_kaydet(
    model_large,
    tokenizer_large,
    veri_kumesi_gpt4o,
    gpt2_large_kaydetme_dizini_gpt4o,
)

## GPT-2 Large modelini DeepSeek veri kümesi ile eğit

In [ ]:
# DeepSeek veri kümesiyle Large modeli eğit
print("\nGPT-2 Large modeli DeepSeek veri kümesi ile eğitiliyor...")
model_large, tokenizer_large = model_ve_tokenizer_yukle(gpt2_large_model_adi)
model_egit_ve_kaydet(
    model_large,
    tokenizer_large,
    veri_kumesi_deepseek,
    gpt2_large_kaydetme_dizini_deepseek,
)